# Coleta de dados

In [44]:
# IMPORTAÇÃO DAS PRINCIPAIS BIBLIOTECAS PARA FUNCIONAMENTO DOS MODELOS:

In [45]:
import pandas as pd
import numpy as np
# import nltk
# import re
# from nltk.corpus import stopwords
# from nltk.stem import RSLPStemmer
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
# nltk.download('rslp')
# nltk.download('stopwords')
from  componetes_preprocessamento import RemoveStopWords, Cleaner, Tokenizador, Stemmer, Joiner, pega_resultados,salvando_em_arquivo
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [46]:
# !conda install nltk
# import nltk
# nltk.download_gui()

In [47]:
dataset = pd.read_csv("datasets/reviews.csv")

# Pré-Processamento dos Dados

## Definindo Tipos

In [48]:
dataset.head(5)

,Unnamed: 0,order_id,review_id,review_score,review_comment_message
0,3,658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,1,Recebi bem antes do prazo estipulado.
1,4,8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,1,Parabéns lojas lannister adorei comprar pela I...
2,9,b9bf720beb4ab3728760088589c62129,8670d52e15e00043ae7de4c01cc2fe06,0,aparelho eficiente. no site a marca do aparelh...
3,12,9d6f15f95d01e79bd1349cc208361f09,4b49719c8a200003f700d3d986ea1a19,0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,15,e51478e7e277a83743b6f9991dbfa3fb,3948b09f7c818e2d86c9a546758b2335,1,"Vendedor confiável, produto ok e entrega antes..."


In [49]:
X = dataset["review_comment_message"].copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 199)

In [51]:
resultados = []

# Accuracy

In [31]:
modelo = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(1,26,2),
              'metric': ['euclidean', 'manhattan', 'minkowski']
              }
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("KNN", GridSearchCV(modelo, param_grid, scoring="accuracy", cv=kfold, n_jobs=-1)),
                    ])

pipeline.fit(X_train, y_train)



Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('KNN',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                              estimator=KNeighborsClassifier(), n_jobs=-1,
                              param_grid={'metric': ['euclidean', 'manhattan',
                                                     'minkowski'],
                                          'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                              scoring='accuracy'))])

In [32]:

cvres = pipeline["KNN"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6766278644026232 {'metric': 'euclidean', 'n_neighbors': 11}
0.6766278644026232 {'metric': 'minkowski', 'n_neighbors': 11}
0.675089855110695 {'metric': 'euclidean', 'n_neighbors': 9}
0.675089855110695 {'metric': 'minkowski', 'n_neighbors': 9}
0.6748455517964793 {'metric': 'euclidean', 'n_neighbors': 15}
0.6748455517964793 {'metric': 'minkowski', 'n_neighbors': 15}
0.6748453563929087 {'metric': 'minkowski', 'n_neighbors': 13}
0.6748453563929087 {'metric': 'euclidean', 'n_neighbors': 13}
0.6737619046572242 {'metric': 'minkowski', 'n_neighbors': 7}
0.6737619046572242 {'metric': 'euclidean', 'n_neighbors': 7}
0.6732376002389786 {'metric': 'manhattan', 'n_neighbors': 9}
0.6724335633966124 {'metric': 'manhattan', 'n_neighbors': 11}
0.6721193300295476 {'metric': 'euclidean', 'n_neighbors': 17}
0.6721193300295476 {'metric': 'minkowski', 'n_neighbors': 17}
0.6710704769385927 {'metric': 'manhattan', 'n_neighbors': 13}
0.6706512996413612 {'metric': 'minkowski', 'n_neighbors': 19}
0.6706512996413

In [33]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("knn", "Grid Search", y_test,y_pred , "Acuracia", pipeline["KNN"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.70      0.74      0.72      3208
           0       0.39      0.13      0.19      2918
           1       0.69      0.90      0.78      6137

    accuracy                           0.67     12263
   macro avg       0.60      0.59      0.57     12263
weighted avg       0.62      0.67      0.63     12263

['knn', 'Grid Search', 0.672836989317459, 0.5651545931109594, 0.5965299884579923, 0.5876613405082093, 'Acuracia', {'metric': 'euclidean', 'n_neighbors': 11}]


# F1 - Score

In [34]:
modelo = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(1,26,2),
              'metric': ['euclidean', 'manhattan', 'minkowski']
              }
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("KNN", GridSearchCV(modelo, param_grid, scoring="f1_macro", cv=kfold, n_jobs=-1)),
                    ])

pipeline.fit(X_train, y_train)



Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('KNN',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                              estimator=KNeighborsClassifier(), n_jobs=-1,
                              param_grid={'metric': ['euclidean', 'manhattan',
                                                     'minkowski'],
                                          'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                              scoring='f1_macro'))])

In [35]:

cvres = pipeline["KNN"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.5848737866105851 {'metric': 'euclidean', 'n_neighbors': 5}
0.5848737866105851 {'metric': 'minkowski', 'n_neighbors': 5}
0.5838467868315794 {'metric': 'euclidean', 'n_neighbors': 7}
0.5838467868315794 {'metric': 'minkowski', 'n_neighbors': 7}
0.5832316080798915 {'metric': 'manhattan', 'n_neighbors': 5}
0.5796136955220674 {'metric': 'euclidean', 'n_neighbors': 9}
0.5796136955220674 {'metric': 'minkowski', 'n_neighbors': 9}
0.5748615941796005 {'metric': 'minkowski', 'n_neighbors': 11}
0.5748615941796005 {'metric': 'euclidean', 'n_neighbors': 11}
0.5736275961473595 {'metric': 'manhattan', 'n_neighbors': 7}
0.5728351044180846 {'metric': 'minkowski', 'n_neighbors': 3}
0.5728351044180846 {'metric': 'euclidean', 'n_neighbors': 3}
0.5718421235508391 {'metric': 'manhattan', 'n_neighbors': 3}
0.5698293946595634 {'metric': 'manhattan', 'n_neighbors': 9}
0.5678653538252896 {'metric': 'euclidean', 'n_neighbors': 13}
0.5678653538252896 {'metric': 'minkowski', 'n_neighbors': 13}
0.5657877418560214 {

In [36]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("knn", "Grid Search", y_test,y_pred , "F1score", pipeline["KNN"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.70      0.75      0.72      3208
           0       0.37      0.19      0.25      2918
           1       0.71      0.85      0.78      6137

    accuracy                           0.67     12263
   macro avg       0.59      0.60      0.58     12263
weighted avg       0.63      0.67      0.64     12263

['knn', 'Grid Search', 0.6671287613145234, 0.5829834250918987, 0.5943353621463708, 0.5968348436829974, 'F1score', {'metric': 'euclidean', 'n_neighbors': 5}]


# Otimizando com o Randomized search:

## Acurácia

In [37]:
from sklearn.model_selection import RandomizedSearchCV

modelo = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(1,26,2),
              'metric': ['euclidean', 'manhattan', 'minkowski']
              }
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("KNN", RandomizedSearchCV(modelo, param_grid, cv=kfold, scoring='accuracy', n_iter=10, n_jobs=-1 )),
                    
                    ])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('KNN',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                                    estimator=KNeighborsClassifier(), n_jobs=-1,
                                    param_distributions={'metric': ['euclidean',
                                                                    'manhattan',
                                                                    'minkowski'],
                                                         'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                                    scoring='accuracy'))])

In [38]:
cvres = pipeline["KNN"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6748455517964793 {'n_neighbors': 15, 'metric': 'minkowski'}
0.6748453563929087 {'n_neighbors': 13, 'metric': 'minkowski'}
0.6737619046572242 {'n_neighbors': 7, 'metric': 'minkowski'}
0.6706512996413612 {'n_neighbors': 19, 'metric': 'minkowski'}
0.6689384530045863 {'n_neighbors': 7, 'metric': 'manhattan'}
0.667540694625498 {'n_neighbors': 19, 'metric': 'manhattan'}
0.6669812664154264 {'n_neighbors': 5, 'metric': 'euclidean'}
0.6669812664154264 {'n_neighbors': 5, 'metric': 'minkowski'}
0.6664920125126677 {'n_neighbors': 21, 'metric': 'manhattan'}
0.6203907656180578 {'n_neighbors': 1, 'metric': 'minkowski'}


In [39]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("knn", "Randomized Search", y_test, y_pred, "Acuracia", pipeline["KNN"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.70      0.73      0.72      3208
           0       0.43      0.10      0.16      2918
           1       0.68      0.92      0.78      6137

    accuracy                           0.67     12263
   macro avg       0.61      0.58      0.55     12263
weighted avg       0.63      0.67      0.62     12263

['knn', 'Randomized Search', 0.6747940960613227, 0.5530896659254466, 0.6055195719979736, 0.582619710640465, 'Acuracia', {'n_neighbors': 15, 'metric': 'minkowski'}]


## F1 score

In [40]:
modelo = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(1,26,2),
              'metric': ['euclidean', 'manhattan', 'minkowski']
              }
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("KNN", RandomizedSearchCV(modelo, param_grid, cv=kfold, scoring="f1_macro", n_iter=10, n_jobs=-1 )),
                    
                    ])

pipeline.fit(X_train, y_train)

C:\Users\arm4\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [       nan 0.56578774 0.56550262 0.5796137  0.5796137  0.56786535
 0.55131506 0.58487379 0.54240695 0.54240695]
  warnings.warn(


Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('KNN',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                                    estimator=KNeighborsClassifier(), n_jobs=-1,
                                    param_distributions={'metric': ['euclidean',
                                                                    'manhattan',
                                                                    'minkowski'],
                                                         'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                                    scoring='f1_macro'))])

In [41]:
cvres = pipeline["KNN"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

nan {'n_neighbors': 5, 'metric': 'euclidean'}
0.5848737866105851 {'n_neighbors': 5, 'metric': 'minkowski'}
0.5796136955220674 {'n_neighbors': 9, 'metric': 'euclidean'}
0.5796136955220674 {'n_neighbors': 9, 'metric': 'minkowski'}
0.5678653538252896 {'n_neighbors': 13, 'metric': 'euclidean'}
0.5657877418560214 {'n_neighbors': 1, 'metric': 'euclidean'}
0.5655026176413898 {'n_neighbors': 1, 'metric': 'manhattan'}
0.5513150645039735 {'n_neighbors': 17, 'metric': 'manhattan'}
0.542406946939024 {'n_neighbors': 23, 'metric': 'euclidean'}
0.542406946939024 {'n_neighbors': 23, 'metric': 'minkowski'}


In [42]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("knn", "Randomized Search", y_test, y_pred, "F1score", pipeline["KNN"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.70      0.75      0.72      3208
           0       0.37      0.19      0.25      2918
           1       0.71      0.85      0.78      6137

    accuracy                           0.67     12263
   macro avg       0.59      0.60      0.58     12263
weighted avg       0.63      0.67      0.64     12263

['knn', 'Randomized Search', 0.6671287613145234, 0.5829834250918987, 0.5943353621463708, 0.5968348436829974, 'F1score', {'n_neighbors': 5, 'metric': 'minkowski'}]


In [43]:
salvando_em_arquivo("resultados/KNN_resultados.csv", resultados)